In [4]:
from kubernetes import client, config
import json

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()
v1 = client.ApiextensionsV1Api()

In [54]:
definition = v1.read_custom_resource_definition('authorizationpolicies.security.istio.io')
definition

{'api_version': 'apiextensions.k8s.io/v1',
 'kind': 'CustomResourceDefinition',
 'metadata': {'annotations': {'helm.sh/resource-policy': 'keep',
                              'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"apiextensions.k8s.io/v1","kind":"CustomResourceDefinition","metadata":{"annotations":{"helm.sh/resource-policy":"keep"},"labels":{"app":"istio-pilot","chart":"istio","heritage":"Tiller","install.operator.istio.io/owning-resource-namespace":"istio-system","istio":"security","istio.io/rev":"default","operator.istio.io/component":"Base","operator.istio.io/managed":"Reconcile","operator.istio.io/version":"1.13.3","release":"istio"},"name":"authorizationpolicies.security.istio.io"},"spec":{"group":"security.istio.io","names":{"categories":["istio-io","security-istio-io"],"kind":"AuthorizationPolicy","listKind":"AuthorizationPolicyList","plural":"authorizationpolicies","singular":"authorizationpolicy"},"scope":"Namespaced","versions":[{"name":"v1beta1",

In [8]:
api = client.CustomObjectsApi()

In [47]:
params = dict(
  group ='security.istio.io',
  version = 'v1beta1',
  namespace = 'istio-system',
  plural='authorizationpolicies',
  name = "ingress-policy")
block_ips_policy = api.get_namespaced_custom_object(**params)

In [48]:
block_ips_policy['spec']['rules'][0]['from'][0]['source']['remoteIpBlocks'].append('192.168.1.11')

In [49]:
block_ips_policy['spec']['rules'][0]['from'][0]['source']['remoteIpBlocks']

['56.5.6.7', '72.9.5.6', '98.1.2.3', '10.244.1.1']

In [51]:
patch_params = dict(body = block_ips_policy, **params)
api.patch_namespaced_custom_object(**patch_params)

{'apiVersion': 'security.istio.io/v1beta1',
 'kind': 'AuthorizationPolicy',
 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"security.istio.io/v1beta1","kind":"AuthorizationPolicy","metadata":{"annotations":{},"name":"ingress-policy","namespace":"istio-system"},"spec":{"action":"ALLOW","rules":[{"from":[{"source":{"remoteIpBlocks":["56.5.6.7","72.9.5.6","98.1.2.3"]}}]}],"selector":{"matchLabels":{"app":"istio-ingressgateway"}}}}\n'},
  'creationTimestamp': '2022-05-10T17:03:16Z',
  'generation': 18,
  'managedFields': [{'apiVersion': 'security.istio.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:kubectl.kubernetes.io/last-applied-configuration': {}}},
     'f:spec': {'.': {},
      'f:selector': {'.': {}, 'f:matchLabels': {'.': {}, 'f:app': {}}}}},
    'manager': 'kubectl-client-side-apply',
    'operation': 'Update',
    'time': '2022-05-15T17:13:21Z'},
   {'apiVersion': '